In [13]:
import pandas as pd
import numpy as np

In [14]:
hotel = pd.read_csv("/Users/laminthiri/Desktop/SP-1/Group5_code/HOTEL_OUTDATASET.CSV")
user = pd.read_csv("/Users/laminthiri/Desktop/SP-1/Group5_code/UserData.csv")


In [15]:
# Normalize column names
hotel.columns = hotel.columns.str.strip().str.lower()
user.columns = user.columns.str.strip().str.lower()

# Make sure IDs are strings
hotel["itemid"] = hotel["itemid"].astype(str)
user["userid"] = user["userid"].astype(str)
user["itemid"] = user["itemid"].astype(str)





In [16]:
def clean_text(text):
    text = str(text).lower()
    for ch in ["[", "]", "&", "+", "-", "/", "(", ")", ",", ".", ":", "|"]:
        text = text.replace(ch, " ")
    words = text.split()
    stopwords = {"or","and","with","the","of","a","an","to","for","in","at","on","by"}
    return set(w for w in words if w and w not in stopwords)

def semicolon_split_tokens(s):
    toks = set()
    for part in str(s).split(";"):
        toks |= clean_text(part)
    return toks


In [17]:
hotel["roomtype"] = hotel.get("roomtype", "")
hotel["roomamenities"] = hotel.get("roomamenities", "")

hotel["roomtype_tokens"] = hotel["roomtype"].fillna("").apply(clean_text)
hotel["amenity_tokens"]  = hotel["roomamenities"].fillna("").apply(semicolon_split_tokens)
hotel["item_tokens"]     = hotel.apply(lambda r: r["roomtype_tokens"] | r["amenity_tokens"], axis=1)

# Dictionary: itemid -> token set
item_to_tokens = dict(zip(hotel["itemid"], hotel["item_tokens"]))


item_to_tokens

{'1': {'air',
  'alarm',
  'all',
  'area',
  'bar',
  'bathrobes',
  'box',
  'channels',
  'clock',
  'closet',
  'coffeetea',
  'comfort',
  'conditioning',
  'daily',
  'demand',
  'desk',
  'detector',
  'double',
  'dryer',
  'facilities',
  'fi',
  'free',
  'hair',
  'heating',
  'ironing',
  'laptop',
  'linens',
  'maker',
  'mini',
  'mirror',
  'movies',
  'newspaper',
  'press',
  'room',
  'rooms',
  'safe',
  'satellitecable',
  'seating',
  'service',
  'smoke',
  'soundproofing',
  'telephone',
  'toiletries',
  'towels',
  'trouser',
  'up',
  'wake',
  'wi'},
 '2': {'air',
  'alarm',
  'all',
  'area',
  'bar',
  'bathrobes',
  'box',
  'channels',
  'classic',
  'clock',
  'closet',
  'coffeetea',
  'conditioning',
  'daily',
  'desk',
  'detector',
  'double',
  'dryer',
  'facilities',
  'fi',
  'free',
  'hair',
  'heating',
  'ironing',
  'laptop',
  'linens',
  'maker',
  'mini',
  'mirror',
  'newspaper',
  'press',
  'room',
  'rooms',
  'safe',
  'satellitec

In [18]:
user_ids = user["userid"].drop_duplicates().iloc[:5].tolist()
profiles_rows = []
user_profile_tokens = {}

for uid in user_ids:
    visited = user.loc[user["userid"] == uid, "itemid"].tolist()
    toks = set().union(*(item_to_tokens.get(i, set()) for i in visited)) if visited else set()
    user_profile_tokens[uid] = toks
    profiles_rows.append({
        "userid": uid,
        "num_visited_hotels": len(visited),
        "profile_token_count": len(toks),
        "profile_tokens": " | ".join(sorted(toks))   # safe delimiter
    })

user_profiles_df = pd.DataFrame(profiles_rows).set_index("userid")
user_profiles_df

,num_visited_hotels,profile_token_count,profile_tokens
userid,,,
1,4,60,air | all | area | bar | bathrobes | bathtub |...
2,3,80,2 | additional | adults | air | alarm | all | ...
3,3,74,additional | air | alarm | all | apartment | a...
4,3,52,air | all | area | bar | blackout | bottled | ...
5,3,61,additional | air | all | area | bar | bed | bl...


In [19]:
user_profiles_df.to_csv("Profile..csv", index=False)

In [20]:
all_item_ids = hotel["itemid"].tolist()
similarity_matrix = pd.DataFrame(np.nan, index=user_profiles_df.index, columns=all_item_ids, dtype=float)

for uid in user_profiles_df.index:
    user_tokens = user_profile_tokens.get(uid, set())
    visited_items = set(user.loc[user["userid"] == uid, "itemid"])
    for itemid in all_item_ids:
        if itemid in visited_items:
            continue  # skip visited
        hotel_tokens = item_to_tokens.get(itemid, set())
        union = user_tokens | hotel_tokens
        sim = len(user_tokens & hotel_tokens) / len(union) if union else 0.0
        similarity_matrix.loc[uid, itemid] = sim
similarity_matrix

,1,2,3,4,5,6,7,8,9,10,...,771,772,773,774,775,776,777,778,779,780
userid,,,,,,,,,,,,,,,,,,,,,
1,0.565217,0.573529,0.588235,0.709677,0.400000,0.412698,0.406250,0.409091,0.412698,0.171875,...,0.560976,0.413333,0.430769,0.449275,0.338462,0.343750,0.312500,0.322581,0.403226,0.403226
2,0.488372,0.494118,0.505882,0.415730,0.247191,0.238636,0.235955,0.255556,0.238636,0.130952,...,0.827160,0.415730,0.329412,0.379310,0.273810,0.261905,0.238095,0.259259,0.304878,0.304878
3,0.525000,0.531646,0.544304,0.463415,0.250000,0.256098,0.253012,0.244186,0.256098,0.126582,...,NaN,0.445783,0.354430,0.390244,0.278481,0.298701,0.256410,0.263158,NaN,0.346667
4,0.587302,0.596774,0.612903,0.580645,0.406780,0.396552,0.366667,0.370968,0.396552,0.196429,...,0.518987,0.400000,0.416667,0.460317,0.410714,0.392857,0.357143,0.396226,0.436364,0.436364
5,0.651515,0.611940,0.651515,0.646154,0.415385,0.406250,0.400000,0.424242,0.428571,0.169231,...,0.517647,0.426667,0.468750,0.463768,0.353846,0.338462,0.307692,0.338710,0.396825,0.396825


In [21]:
similarity_matrix.to_csv("Jaccard_SimMatrix.csv")

In [22]:
sim_long = (
    similarity_matrix
    .stack(dropna=True)
    .rename("Similarity")
    .reset_index()
    .rename(columns={"level_0": "userid", "level_1": "itemid"})
)

# Build deterministic item->meta lookup once (stable first row per itemid)
meta = (
    hotel.sort_values(["itemid", "hotelid", "hotelname"], kind="stable")
         .drop_duplicates("itemid", keep="first")[["itemid", "hotelid", "hotelname"]]
)
itemid_to_hotelid   = meta.set_index("itemid")["hotelid"]
itemid_to_hotelname = meta.set_index("itemid")["hotelname"]

# Map (no merge ambiguity, no accidental math)
sim_long["Hotel ID"]   = sim_long["itemid"].map(itemid_to_hotelid)
sim_long["Hotel Name"] = sim_long["itemid"].map(itemid_to_hotelname)

# -----------------------
# 6) Top-5 per user and save
# -----------------------
top_recommendations = (
    sim_long
    .sort_values(["userid", "Similarity", "itemid"], ascending=[True, False, True])
    .groupby("userid", as_index=False, sort=False)
    .head(5)
    .reset_index(drop=True)
)

out = top_recommendations.rename(columns={
    "userid": "User ID",
    "itemid": "Item ID"
})[["User ID", "Item ID", "Hotel ID", "Hotel Name", "Similarity"]]

out

/var/folders/n1/557rrt693zb34vlddb1q712h0000gn/T/ipykernel_83932/2022661689.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack(dropna=True)


,User ID,Item ID,Hotel ID,Hotel Name,Similarity
0,1,140,90896,Hotel Altstadt Vienna,0.714286
1,1,4,31,The Harmonie Vienna BW Premier Collection,0.709677
2,1,83,70993,Star Inn Hotel Salzburg Airport-Messe by Comfort,0.677419
3,1,84,70993,Star Inn Hotel Salzburg Airport-Messe by Comfort,0.677419
4,1,586,503028,Sporthotel Igls,0.672131
5,2,769,7482881,Pension Restaurant Rosenberg,0.888889
6,2,771,7482881,Pension Restaurant Rosenberg,0.827160
7,2,747,6881583,Hotel Die Residenz Superior,0.694118
8,2,748,6881583,Hotel Die Residenz Superior,0.682353
9,2,749,6881583,Hotel Die Residenz Superior,0.682353


In [23]:
out.to_csv("Recommendation.csv", index=False)

In [24]:
def clean_text(text):
    text = text.lower()
    
    for ch in [";", "[", "]", "&", "+"]:
        text = text.replace(ch, " ")
    
    words = text.split()
    
    stopwords = {"or", "and", "with", "the", "of", "-"}
    cleaned_words = []
    for w in words:
        if w not in stopwords:
            cleaned_words.append(w)

    return " ".join(cleaned_words)


In [25]:
roomamenities = hotel["roomamenities"]
roomamenities_docs = []
for r in roomamenities:
    roomamenities_docs.append(clean_text(r).split())
roomamenities_docs

[['air',
  'conditioning',
  'alarm',
  'clock',
  'bathrobes',
  'closet',
  'coffeetea',
  'maker',
  'daily',
  'newspaper',
  'desk',
  'free',
  'wi-fi',
  'in',
  'all',
  'rooms',
  'hair',
  'dryer',
  'heating',
  'in-room',
  'safe',
  'box',
  'ironing',
  'facilities',
  'laptop',
  'safe',
  'box',
  'linens',
  'mini',
  'bar',
  'mirror',
  'on-demand',
  'movies',
  'satellitecable',
  'channels',
  'seating',
  'area',
  'smoke',
  'detector',
  'soundproofing',
  'telephone',
  'toiletries',
  'towels',
  'trouser',
  'press',
  'wake-up',
  'service'],
 ['air',
  'conditioning',
  'alarm',
  'clock',
  'bathrobes',
  'closet',
  'coffeetea',
  'maker',
  'daily',
  'newspaper',
  'desk',
  'free',
  'wi-fi',
  'in',
  'all',
  'rooms',
  'hair',
  'dryer',
  'heating',
  'in-room',
  'safe',
  'box',
  'ironing',
  'facilities',
  'laptop',
  'safe',
  'box',
  'linens',
  'mini',
  'bar',
  'mirror',
  'satellitecable',
  'channels',
  'seating',
  'area',
  'smoke'

In [26]:
#tokens in roomtype
roomtype = hotel["roomtype"]
roomtype_docs = []
for r in roomtype: 
    roomtype_docs.append(clean_text(r).split())
roomtype_docs

[['comfort', 'double', 'room'],
 ['classic', 'double', 'twin', 'room'],
 ['superior', 'double', 'room'],
 ['suite'],
 ['classic', 'double', 'twin', 'room'],
 ['comfort', 'room'],
 ['run', 'house'],
 ['suite-1', 'double', 'bed', 'living', 'room'],
 ['superior', 'room'],
 ['double', 'non', 'smoking'],
 ['superior', 'double', 'room'],
 ['run', 'house'],
 ['suite', 'balcony'],
 ['premium', 'double', 'room'],
 ['comfort', 'double', 'room'],
 ['double', 'twin', 'standard'],
 ['double', 'single', 'use', 'standard'],
 ['superior', 'double', 'room'],
 ['family', 'room'],
 ['family', 'room', '2', 'adults', '2', 'children'],
 ['executive', 'room'],
 ['comfort', 'room'],
 ['run', 'house'],
 ['comfort', 'triple', 'room'],
 ['business', 'single'],
 ['double'],
 ['single'],
 ['single', 'comfort'],
 ['single', 'executive', 'superior', 'room', 'only'],
 ['single', 'room', 'executive'],
 ['triple'],
 ['double', 'single', 'use'],
 ['executive', 'superior', 'room'],
 ['room'],
 ['double', 'room'],
 ['doub

In [27]:
itemid = hotel["itemid"]
#Creating item feature vector
item_features = {}
for i in itemid:
    item_features[i] = (
    roomtype_docs[i-1] +
    roomamenities_docs[i-1] 
)
    
item_features

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
tokens = []
for token in item_features.values():
    tokens.extend(token)

tokens = set(tokens)

tokens
print(len(tokens))

In [ ]:
total_docs_with_t = {}
for token in tokens:
    count = 0
    for item in item_features.values():
        if token in item:
            count += 1
    total_docs_with_t[token] = count
total_docs_with_t


In [ ]:
total_words = sum(total_docs_with_t.values())
total_words

In [ ]:
total_docs = len(hotel)
total_docs

In [ ]:
idf_scores = {}
for term,doc_freq in total_docs_with_t.items():
    idf = math.log(total_docs / doc_freq)
    idf_scores[term] = idf

idf_scores

In [ ]:
tf_idf = {}

for item, terms in item_features.items():
    tf_idf[item] = {}
    total_terms = len(terms)

    for word in set(terms):
        tf = terms.count(word) / total_terms
        tf_idf[item][word] = tf * idf_scores[word]

tf_idf

In [ ]:
import csv

tokens = sorted(tokens)
header = ["item_id"] + tokens

rows = []

for item_id, word_scores in tf_idf.items():
    row = [item_id]
    for word in tokens:
        row.append(word_scores.get(word, 0.0))  
    rows.append(row)

with open("feature_vectors.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)


In [ ]:
user.columns = user.columns.str.strip()

In [ ]:
merged = pd.merge(user, hotel, on="itemid", how="inner")
merged.head()

In [ ]:
item_vectors = pd.read_csv("feature_vectors.csv")
item_vectors.head()

In [ ]:
user_dic = user.groupby("userid")["itemid"].apply(list).to_dict()
user_dic

In [ ]:
features = pd.read_csv("feature_vectors.csv")

features = features.set_index("item_id")

user_profiles = {}

for uid, items in user_dic.items():
    vectors = features.loc[items]
    summed = vectors.sum(axis=0)
    averaged = vectors.mean(axis=0)
    user_profiles[uid] = averaged
    
user_profiles

In [ ]:
header = ["user_id"] + tokens

rows = []

for uid, word_scores in user_profiles.items():
    row = [uid]
    for word in tokens:
        row.append(word_scores.get(word, 0.0))  
    rows.append(row)

with open("Profile_2.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)


In [ ]:
#create unseen items for each user
unseen_items = {}
for uid, seen in user_dic.items():
    unseen_items[uid] = features.drop(index=seen)

unseen_items[1]

In [ ]:
# Calculate cosine similarity
def cosine_similarity(user_vec, item_vecs):
    similarities = []
    for item_vec in item_vecs:
        dot_product = sum(u * v for u, v in zip(user_vec, item_vec))
        user_norm = np.sqrt(sum(u ** 2 for u in user_vec))
        item_norm = np.sqrt(sum(v ** 2 for v in item_vec))
        if user_norm == 0 or item_norm == 0:
            similarity = 0.0
        else:
            similarity = dot_product / (user_norm * item_norm)
        similarities.append(similarity)
    return similarities

# Calculate similarity scores for unseen items and store in 'score' column
unseen_items_scored = {}
for uid, unseen_df in unseen_items.items():
    user_vec = user_profiles[uid].values.tolist()
    item_vecs = unseen_df.values.tolist()
    scores = cosine_similarity(user_vec, item_vecs)
    unseen_df_scored = unseen_df.copy()
    unseen_df_scored['similarity_score'] = scores
    unseen_items_scored[uid] = unseen_df_scored

unseen_items_scored[1]

In [ ]:
# Combine all users' unseen_items_scored into one dataframe with 'user_id' column
dfs = []
for uid, df in unseen_items_scored.items():
    df_copy = df.copy()
    df_copy.insert(0, 'user_id', uid)  # Insert 'user_id' as first column
    dfs.append(df_copy.reset_index())  # Reset index to bring 'item_id' as a column

unseen_items_scored_combined = pd.concat(dfs)
unseen_items_scored_combined

In [ ]:
# Pivot similarity scores: items as rows, users as columns
pivot_scores = unseen_items_scored_combined.pivot_table(
    index='item_id', columns='user_id', values='similarity_score'
)

pivot_scores.to_csv("CosineSim.csv", sep='\t')


In [ ]:
# Show top 10 unseen items with the highest score for each user
top_n = 10
top_recommendations = {}
for uid, df in unseen_items_scored.items():
    top_items = df.sort_values(by='similarity_score', ascending=False).head(top_n)
    top_recommendations[uid] = top_items
    print(f"User {uid} top {top_n} recommendations:")
    print(top_items)
    print("\n---\n")

In [ ]:
# Show only item_id and score columns for top 10 recommendations for each user
user_top10 = {}
for uid, df in top_recommendations.items():
    user_top10[uid] = df[['similarity_score']].reset_index()[['item_id', 'similarity_score']]
    print(f"User {uid} top {top_n} recommendations (item_id and similarity_score):")
    print(user_top10[uid])
    print("\n---\n")

In [ ]:
# Join users_top10 to hotel by item_id to get hotel details for top recommendations
users_top10_with_hotel = pd.merge(users_top10, hotel, left_on='item_id', right_on='itemid', how='left')
users_top10_with_hotel.head()

In [ ]:
# Show only user_id, item_id, hotelid, hotelname, similarity_score columns
users_top10_with_hotel = users_top10_with_hotel[['user_id', 'item_id', 'hotelid', 'hotelname', 'similarity_score']]
users_top10_with_hotel

In [ ]:
users_top10_with_hotel.to_csv("Recommendation2.csv", index=False)